> 🚨 This Notebook is work in progress. 

# Groq: Generative Information Density Experiment 🍞 (GrID) 

This notebook services as our experiments in understanding "information density" — as a concept, topic of research, practical tool and natural occurence. We start the experiment with no external knowledge, simply an intuition of what "information density" means to us. From there, we'll gradually evolve our approach by tapping into research literature and existing tools. At the end, we'll present one compact solution that compiles all the findings in the notebook. 👾


# GrID: Generative Information Density 👩🏻‍🎤

Let's get started with the architecture. 

### Architecture and Design 🛖


In [125]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

In [126]:
install_requirements()

Installing requirements...
Requirements installed successfully.


In [127]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    load_dotenv()

    GROQ_API_KEY = os.getenv("GROQ_API_KEY")

    if GROQ_API_KEY is None:
        print("Please set the GROQ_API_KEY environment variable.")
        exit(1)
    else:
        print("GROQ_API_KEY is set.")

In [128]:
setup_env()

GROQ_API_KEY is set.


In [129]:
from pydantic import BaseModel
from enum import Enum
from typing import List

class Sentiment(Enum):
    positive = "positive"
    negative = "negative"
    neutral = "neutral"

class StrengthMetric(BaseModel):
    """
    The strength of the metric in the text.
    """
    strength: float
    confidence: float

class TopicScoreNode(StrengthMetric):
    """
    Extracted topic with strength and confidence. 
    The topic strength indicates how strongly the topic is present in the text.
    The confidence indicates how confident the model is in the topic extraction and strength.
    """
    topic: str

class QuestionNode(StrengthMetric):
    """
    Extracted question with strength and confidence. 
    The question strength indicates how strongly the question is present in the text.
    The confidence indicates how confident the model is in the question extraction and strength.
    """
    question: str

class EmbeddedMeaningNode(StrengthMetric):
    """
    Extracted embedded meaning with strength and confidence. 
    The embedded meaning strength indicates how strongly the embedded meaning is present in the text.
    The confidence indicates how confident the model is in the embedded meaning extraction and strength.
    """
    embedded_meaning: str

class TangentThoughtNode(StrengthMetric):
    """
    Extracted tangent thought with strength and confidence. 
    The tangent thought strength indicates how strongly the tangent thought is present in the text.
    The confidence indicates how confident the model is in the tangent thought extraction and strength.
    """
    tangent_thought: str

class TopicScores(BaseModel):
    """
    Extracted topics from the strength with their confidence and strengths. 
    """
    topic_scores: List[TopicScoreNode]

class QuestionScores(BaseModel):
    """
    Extracted questions from the strength with their confidence and strengths. 
    """
    question_scores: List[QuestionNode]

class EmbeddedMeaningScores(BaseModel):
    """
    Extracted embedded meanings from the strength with their confidence and strengths. 
    """
    embedded_meaning_scores: List[EmbeddedMeaningNode]

class SentimentScore(BaseModel):
    """
    The sentiment score of the text.
    """
    sentiment: Sentiment
    confidence: float


class TangentThoughtScores(BaseModel):
    """
    Extracted tangent thoughts from the strength with their confidence and strengths. 
    """
    tangent_thought_scores: List[TangentThoughtNode]


class PreliminaryAnalysis(BaseModel):
    """
    Preliminary analysis of the text.
    """
    topic_scores: TopicScores
    question_scores: QuestionScores
    embedded_meaning_scores: EmbeddedMeaningScores
    sentiment_score: SentimentScore
    tangent_thought_scores: TangentThoughtScores

In [130]:
import instructor
from groq import Groq
import traceback
from pydantic import BaseModel
from typing import Union
from openai import OpenAI

DEFAULT_MODEL = "llama-3.3-70b-versatile"
DEFAULT_OPENAI_MODEL = "gpt-4o"

class LLMErrorResponse(BaseModel):
    error: str


def get_groq_client():
    """Returns an instance of the Groq class"""
    groq = Groq(api_key=os.getenv("GROQ_API_KEY"))
    client = instructor.from_groq(groq, mode=instructor.Mode.JSON)
    return client

def get_openai_client():
    """Returns an instance of the OpenAI class"""
    openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    return openai


def llm(
    prompt: str,
    response_model: BaseModel,
    system="You are a helpful AI assistant. The user will talk to you and its your job to provide detailed and clear responses.",
    model=DEFAULT_MODEL,
) -> Union[BaseModel, LLMErrorResponse]:
    """Calls LLM API with the given prompt. Defaults to llama-3.3-70b-versatile""",
    messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]
    try:
        client = get_groq_client()
        response = client.chat.completions.create(
            messages=messages, model=model, response_model=response_model
        )
        return response
    except Exception as e:
        try:
            openai = get_openai_client()
            completion = openai.beta.chat.completions.parse(messages=messages, model=DEFAULT_OPENAI_MODEL, response_format=response_model)
            return completion.choices[0].message.parsed
        except Exception as e:
            traceback.print_exc()
            return LLMErrorResponse(error=str(e))

In [131]:
class GrIDEngineNucleus():
    """GrID Engine: Computing information density."""

    def __init__(self, model = "llama-3.3-70b-versatile", verbose = False):
        self.model = model 
        self.verbose = verbose
        

    def get_sentiment(self, text) -> SentimentScore:
        prompt = f"""
        Compute the sentiment of the text.
        Respond with the sentiment and confidence.
        Text: {text}
        """
        response = llm(prompt=prompt, response_model=SentimentScore)
        return response
    
    def get_embedded_meaning(self, text) -> EmbeddedMeaningScores:
        prompt = f"""
        Extract the embedded meanings from the text.
        Respond with the embedded meanings, their strengths and confidence.
        Text: {text}
        """
        response = llm(prompt=prompt, response_model=EmbeddedMeaningScores)
        return response
    
    def get_topic_scores(self, text) -> TopicScores:
        prompt = f"""
        Extract the topics from the text.
        Respond with the topics, their strengths and confidence.
        Text: {text}
        """
        response = llm(prompt=prompt, response_model=TopicScores)
        return response
    
    def get_question_scores(self, text) -> QuestionScores:
        prompt = f"""
        Extract the questions from the text.
        Respond with the questions, their strengths and confidence.
        Text: {text}
        """
        response = llm(prompt=prompt, response_model=QuestionScores)
        return response
    
    def get_tangent_thought_scores(self, text) -> TangentThoughtScores:
        prompt = f"""
        Extract the tangent thoughts from the text.
        Respond with the tangent thoughts, their strengths and confidence.
        Text: {text}
        """
        response = llm(prompt=prompt, response_model=TangentThoughtScores)
        return response
    
    def get_preliminary_analysis(self, text: str, collapse_layers = True) -> PreliminaryAnalysis:
        """Computes the preliminary analysis of the text needed for the GrID engine."""
        if collapse_layers:
            prompt = f"""
                Generate the preliminary analysis of the text.
                Respond with the topic scores, question scores, embedded meaning scores, sentiment score and tangent thought scores.
                Topic Scores: Extract the topics from the text.
                Question Scores: Extract the questions from the text.
                Embedded Meaning Scores: Extract the embedded meanings from the text.
                Sentiment Score: Compute the sentiment of the text.
                Tangent Thought Scores: Extract the tangent thoughts from the text.
                Text: {text}
            """
            preliminary_analysis = llm(prompt=prompt, response_model=PreliminaryAnalysis)
            return preliminary_analysis
        sentiment_score = self.get_sentiment(text)
        topic_scores = self.get_topic_scores(text)
        question_scores = self.get_question_scores(text)
        embedded_meaning_scores = self.get_embedded_meaning(text)
        tangent_thought_scores = self.get_tangent_thought_scores(text)
        return PreliminaryAnalysis(
            topic_scores=topic_scores,
            question_scores=question_scores,
            embedded_meaning_scores=embedded_meaning_scores,
            sentiment_score=sentiment_score,
            tangent_thought_scores=tangent_thought_scores
        )
    


In [132]:

class GrIDNaiveResult(BaseModel):
    """The result of the GrID Engine Naive."""
    information_density: float
    explanation: str

class GrIDEngineNaive(GrIDEngineNucleus):
    """GrID Engine: Computing information density using a naive approach."""
    
    def __init__(self,  model = "llama-3.3-70b-versatile", verbose = False):
        super().__init__(model, verbose=verbose)

    def compute_information_density(self, text: str, collapse_layers = False) -> GrIDNaiveResult:
        """Compute the information density of the text."""
        preliminary_analysis = self.get_preliminary_analysis(text, collapse_layers=collapse_layers)
        preliminary_analysis_json = preliminary_analysis.model_dump_json()
        system = """
            You are GrID Engine Naive. 
            GrID Engine stands for 'Generative Information Density' Engine.
            We have computed a preliminary analysis for a piece of text. 
            Your job is to understand that text, and compute the information density between 0 and 1.
            To assist you, we have extracted the following information from the text:
            - Topics: The main topics that the text discusses.
            - Questions: The questions in the text which indicates the curiosity of the text.
            - Embedded Meanings: The embedded meanings in the text which are not explicitly stated.
            - Sentiment: The sentiment of the text which indicates the overall emotion of the text.
            - Tangent Thoughts: The tangent thoughts in the text which are not directly related to the main topics.
            These components include strengths and confidence levels.
            Use this information to compute the information density.
            Explain your reasoning for the information density in 4 paragraphs in markdown in detail.
            Respond with the information density and the explanation in markdown.
        """

        prompt = f"""
            Based on the given instructions, analyse the preliminary analysis and compute the information density of the text.
            Text: {text}
            Preliminary Analysis: {preliminary_analysis_json}
        """

        response = llm(prompt=prompt, system=system, response_model=GrIDNaiveResult)
        return response

    

In [133]:
TEST_INPUT_TEXTS = [
    """
When learning something new in Software Development, it's easy to get caught up in the "consumption cycle". If you're only consuming lectures, courses, blogs and learning material but spending little to no time building things, you won't get far. 

I personally like spending 80% to 90% of my time actively building and architecting / designing / implementing and the remaining 10% / 20% of my time consuming learning material, lecture, trainings. However, it's important that you strive for a balance that works the best for you. Generally 50% consuming, 50% implementing might be a good balance, but it depends on the subject or topic you're learning and the time you have on your hands.

Sometimes, it's important to just learn enough so that you can get started with execution! There's that sweet spot of "genius" which lies between endless thought, planning, research, and the other extreme which is implementation without feedback or learnings or depth. 

It takes a while to figure this out, but as a rule of thumb; keep building, keep shipping and growth will become a natural outcome of the process!
""",
    """
Excited to be featured as an NVIDIA agents partner during Jensen's keynote at CES 🔥

2025 is the year of production knowledge agents. Everything from document research, to automated document extraction/business logic, to report generation. 

This is a great way to kick off the year. Huge shoutout to Laurie Voss + the NVIDIA team (including Daniel Glogowski) for pulling these resources together.

Blueprint: https://lnkd.in/gHFFKrgC
Video: https://lnkd.in/gQi7U6sq
Notebook: https://lnkd.in/ggmkXiYA
Blog: https://lnkd.in/ghfApUs9
""",
    """If you are leading a project, your only responsibility is to ensure it is delivered, whatever it takes. Here are a few pointers that I have followed 

1. avoid being blocked, always find a way out
2. if there is a chance of a delay, communicate early
3. always look for trade-offs and make sure we pick the right one
4. estimate timelines well; good estimation reduces chaos
5. influence others so that they prioritize our tasks
6. always reiterate key details to ensure alignment, there is no such thing as over-communication.

On the technical and execution side, here's what I ensure

1. form a deep understanding and high clarity about the project
2. create a solid plan, reduce ambiguity, and keep the team focused 
3. be agile, monitor progress, revise plan if required
4. make sure every single person involved in the project is aligned

Delivering a project requires very high focus, clarity, and persistence. Keep the big picture in mind, but execute with attention to detail.

Even if you are early in your career, follow the above, and earn some leadership brownie points.

ps: enrollments open for sys design cohort - arpitbhayani.me/course

hashtag#AsliEngineering hashtag#CareerGrowth""",
    """Yes, materials provide a chip’s physical foundation and the substance of more powerful and compact components. But they are also integral to the advanced fabrication methods and novel chip designs that underpin the industry’s rapid progress in recent decades.

For this reason, materials science is taking on a heightened importance as we grapple with the limits of miniaturization. Advanced materials are needed more than ever for the industry to unlock the new designs and technologies capable of increasing chip efficiency, speed, and power. We are seeing novel chip architectures that embrace the third dimension and stack layers to optimize surface area usage while lowering energy consumption. The industry is harnessing advanced packaging techniques, where separate “chiplets” are fused with varying functions into a more efficient, powerful single chip. This is called heterogeneous integration.""",
    """
    Data science is a good field. 
    It has a lot of potential for career growth
""",
]

In [134]:
from IPython.display import Markdown
from json import dumps
from pydantic import BaseModel

class FormattedResponse(BaseModel):
    content: str


def convert_markdown_to_text(markdown):
    prompt = f"""
    Convert the markdown to plain text.
    Markdown: {markdown}
    """
    response = llm(prompt=prompt, response_model=FormattedResponse)
    return response.content

def format_data_as_markdown_table(data):
    """Formats the data as a markdown table."""
    markdown = "| Text | Information Density |\n"
    markdown += "| --- | --- |\n"
    for item in data:
        trimmed_text = item['text'][:50] + (item['text'][50:] and '...')
        markdown += f"| {trimmed_text} | {item['information_density']} |\n"
    return markdown



def run_naive_grid_engine():
    grid_engine_naive = GrIDEngineNaive(verbose=True)
    results = []
    print(len(TEST_INPUT_TEXTS))
    
    for text in TEST_INPUT_TEXTS[0:2]:
        response = grid_engine_naive.compute_information_density(text)
        results.append({
            "text": text,
            "information_density": response.information_density,
            "explanation": response.explanation
        })
    formatted_tables_markdown = format_data_as_markdown_table(results)
    return Markdown(formatted_tables_markdown)


In [135]:
#run_naive_grid_engine()

In [136]:
input_text = """
Excited to be featured as an NVIDIA agents partner during Jensen's keynote at CES 🔥

2025 is the year of production knowledge agents. Everything from document research, to automated document extraction/business logic, to report generation. 

This is a great way to kick off the year. Huge shoutout to Laurie Voss + the NVIDIA team (including Daniel Glogowski) for pulling these resources together.

Blueprint: https://lnkd.in/gHFFKrgC
Video: https://lnkd.in/gQi7U6sq
Notebook: https://lnkd.in/ggmkXiYA
Blog: https://lnkd.in/ghfApUs9
"""
grid_engine_naive = GrIDEngineNaive(verbose=True)
collapse_layers = False # Set to True to collapse the layers

response = grid_engine_naive.compute_information_density(input_text, collapse_layers=True)

print(f"Text: {input_text}")
print(f"Information Density: {response.information_density}")

Text: 
Excited to be featured as an NVIDIA agents partner during Jensen's keynote at CES 🔥

2025 is the year of production knowledge agents. Everything from document research, to automated document extraction/business logic, to report generation. 

This is a great way to kick off the year. Huge shoutout to Laurie Voss + the NVIDIA team (including Daniel Glogowski) for pulling these resources together.

Blueprint: https://lnkd.in/gHFFKrgC
Video: https://lnkd.in/gQi7U6sq
Notebook: https://lnkd.in/ggmkXiYA
Blog: https://lnkd.in/ghfApUs9

Information Density: 0.85


In [137]:
input_text = """
Excited to be featured as an NVIDIA agents partner during Jensen's keynote at CES 🔥

2025 is the year of production knowledge agents. Everything from document research, to automated document extraction/business logic, to report generation. 

This is a great way to kick off the year. Huge shoutout to Laurie Voss + the NVIDIA team (including Daniel Glogowski) for pulling these resources together.

Blueprint: https://lnkd.in/gHFFKrgC
Video: https://lnkd.in/gQi7U6sq
Notebook: https://lnkd.in/ggmkXiYA
Blog: https://lnkd.in/ghfApUs9
"""
grid_engine_naive = GrIDEngineNaive(verbose=True)
collapse_layers = True # Set to True to collapse the layers

response = grid_engine_naive.compute_information_density(input_text, collapse_layers=True)

print(f"Text: {input_text}")
print(f"Information Density: {response.information_density}")

Text: 
Excited to be featured as an NVIDIA agents partner during Jensen's keynote at CES 🔥

2025 is the year of production knowledge agents. Everything from document research, to automated document extraction/business logic, to report generation. 

This is a great way to kick off the year. Huge shoutout to Laurie Voss + the NVIDIA team (including Daniel Glogowski) for pulling these resources together.

Blueprint: https://lnkd.in/gHFFKrgC
Video: https://lnkd.in/gQi7U6sq
Notebook: https://lnkd.in/ggmkXiYA
Blog: https://lnkd.in/ghfApUs9

Information Density: 0.83


In [138]:
def compare_runtime_with_collapsing(input_text = TEST_INPUT_TEXTS[0]):
    print(f"Input Text: {input_text}")
    import time
    grid_engine_naive = GrIDEngineNaive(verbose=True)
    start_time = time.time()
    response = grid_engine_naive.compute_information_density(input_text, collapse_layers=True)
    print(f"Information Density (no layer collapsing): {response.information_density}")
    end_time = time.time()
    time_with_collapsing = end_time - start_time
    print(f"Time with collapsing: {time_with_collapsing}")

    start_time = time.time()
    response = grid_engine_naive.compute_information_density(input_text, collapse_layers=False)
    end_time = time.time()
    time_without_collapsing = end_time - start_time
    print(f"Information Density (with layer collapsing): {response.information_density}")
    print(f"Time without collapsing: {time_without_collapsing}")

    time_delta_percent = ((time_with_collapsing - time_without_collapsing) / time_with_collapsing) * 100
    print(f"Time difference (collapsing is better by?): {time_delta_percent}%")
    return time_with_collapsing, time_without_collapsing

compare_runtime_with_collapsing()

Input Text: 
When learning something new in Software Development, it's easy to get caught up in the "consumption cycle". If you're only consuming lectures, courses, blogs and learning material but spending little to no time building things, you won't get far. 

I personally like spending 80% to 90% of my time actively building and architecting / designing / implementing and the remaining 10% / 20% of my time consuming learning material, lecture, trainings. However, it's important that you strive for a balance that works the best for you. Generally 50% consuming, 50% implementing might be a good balance, but it depends on the subject or topic you're learning and the time you have on your hands.

Sometimes, it's important to just learn enough so that you can get started with execution! There's that sweet spot of "genius" which lies between endless thought, planning, research, and the other extreme which is implementation without feedback or learnings or depth. 

It takes a while to figur

(13.719287872314453, 25.69143509864807)